In [ ]:
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split

In [ ]:
archivo = "aves.csv" #nombre del archivo
data = pd.read_csv(archivo, #Nombre del archivo
                   index_col=0, #Quito la columna indexada por 0
                  )

In [ ]:
data.head() # Se visualizan los primeros 5 elementos de la data


In [ ]:
data.info() # Se ontiene información relevante del archivo

In [ ]:
data["x1"].value_counts()# Cuenta cuántas veces aparece cada valor único en la columna "x1" 
                        # y devuelve una Serie ordenada de mayor a menor frecuencia.

In [ ]:
data.describe() #Resumen de características de la data

In [ ]:
%matplotlib inline
data.hist(bins=50, figsize=(20,15)) # Se visualiza la data numérica

In [ ]:
corr = data.iloc[:,:-1].corr(method = "pearson")
cmap = sns.diverging_palette(250,354,80,60,center='dark',as_cmap=True)
sns.heatmap(corr, vmax=1, vmin = -0.5, cmap=cmap, square=True, linewidths=0.2)

In [ ]:
# def split_train_test(data, test_radio):
#     '''
#     data: Datos con los cuales se está trabajando
#     test_radio: Porcentaje de la data que se quiere para el test
#     '''
#     shuffle_indices = np.random.permutation(len(data)) #Nos da une permutación de elementos
#     test_set_size = int(len(data)*test_radio) # Porcentaje de datos que obtengo de la data
#     test_indices = shuffle_indices[:test_set_size]
#     train_indices = shuffle_indices[test_set_size:]
#     return data.iloc[train_indices], data.iloc[test_indices] # Devuelve las filas con estos índices

In [ ]:
list(data.columns)

In [ ]:
def calculate_prior(dataFrame, Class):
    classes = sorted(list(dataFrame[Class].unique()))  # Clases presentes en la data
    denominador = len(dataFrame)
    prior = []
    for i in classes:
        numerador = len(dataFrame[dataFrame[Class] == i])  # Número de filas que cumplen la condición
        prior.append(numerador / denominador)  # Probabilidad a priori
    return prior

def verosimilitud_gaussiana(dataFrame, caracteristica, valor_caracteristica, clase, label):
    dataFrame = dataFrame[dataFrame[clase] == label]  # Filtramos los datos de la clase deseada
    mean = dataFrame[caracteristica].mean()  # Media de la característica
    std = dataFrame[caracteristica].std()  # Desviación estándar

    if std == 0:  # Para evitar divisiones por cero
        return 1 if valor_caracteristica == mean else 0

    proba_x_dado_y = (1 / (std * np.sqrt(2 * np.pi))) * np.exp(-((valor_caracteristica - mean) ** 2) / (2 * std ** 2))
    return proba_x_dado_y

def clasificador_ingenuo_bayesiano(dataFrame, X, Y):
    '''
    X: Lista o array de vectores de características
    Y: Nombre de la columna de la clase
    '''
    # Suponemos que todas las columnas excepto Y son características
    # Alternativamente, se podría pasar explícitamente la lista de características.
    caracteristicas = [col for col in dataFrame.columns if col != Y]
    prior = calculate_prior(dataFrame, Y)

    labels = sorted(list(dataFrame[Y].unique()))
    Y_pred = []
    
    for x in X:
        # Inicializar verosimilitud para cada clase
        verosimilitud = [1.0] * len(labels)
        for j in range(len(labels)):
            for i in range(len(caracteristicas)):
                verosimilitud[j] *= verosimilitud_gaussiana(dataFrame, 
                                                             caracteristicas[i], 
                                                             x[i], 
                                                             Y, 
                                                             labels[j])
        # Calcular probabilidad posterior para cada clase
        post_prob = [verosimilitud[j] * prior[j] for j in range(len(labels))]
        
        # Se selecciona la clase con mayor probabilidad
        pred_class_index = np.argmax(post_prob)
        Y_pred.append(labels[pred_class_index])
        
    return np.array(Y_pred)

In [ ]:
train, test = train_test_split(data, test_size=0.2, random_state=41)

X_test = test.iloc[:,:-1].values
Y_test = test.iloc[:, -1].values
Y_predict = clasificador_ingenuo_bayesiano(data, X = X_test, Y = "x2")

In [ ]:
from sklearn.metrics import confusion_matrix, f1_score
print(confusion_matrix(Y_test, Y_predict))
print(f1_score(Y_test, Y_predict))

In [ ]:
import numpy as np

class BernoulliNB:
    def fit(self, X, y):
        """
        Estima parámetros por máxima verosimilitud con suavizado de Laplace.
        """
        self.clases = np.unique(y)
        self.n_clases = self.clases.size
        self.n_atr = X.shape[1]
        n = X.shape[0]

        self.qa = np.zeros((self.n_clases, self.n_atr))
        self.qc = np.zeros(self.n_clases)

        for i, c in enumerate(self.clases):
            Xc = X[np.where(y == c)]
            nc = Xc.shape[0]

            cuentas = np.count_nonzero(Xc, axis=0)
            self.qa[i, :] = (cuentas + 1) / (nc + 2)  # Suavizado de Laplace
            self.qc[i] = nc / n

    def bernoulli_pmf(self, x, p):
        """
        Función de masa de probabilidad de la distribución Bernoulli.
        """
        return p**x * (1 - p)**(1 - x)

    def predict_proba(self, X):
        """
        Calcula la probabilidad a posteriori para cada clase.
        """
        prop = np.zeros((X.shape[0], self.n_clases))
        for i in range(self.n_clases):
            prob_cond = self.bernoulli_pmf(X, self.qa[i, :])
            prop[:, i] = np.prod(prob_cond, axis=1) * self.qc[i]  # Multiplicamos las probabilidades
        return prop

    def predict(self, X):
        """
        Predice las clases del conjunto de datos.
        """
        return np.argmax(self.predict_proba(X), axis=1)

X_ent, X_nuevos, y_ent, y_nuevos = train_test_split(X, y, test_size=0.2, random_state=42)

# Ejemplo de uso
bnb = BernoulliNB()
bnb.fit(X_ent, y_ent)

y_bnb = bnb.predict(X_nuevos)
p_bnb = bnb.predict_proba(X_nuevos)

print(y_bnb)
print(p_bnb)


# Nuevo intento

In [2]:
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split

In [94]:
data.info

<bound method DataFrame.info of             x1         x2    y
0     7.606876  11.421913  0.0
1     8.300079  12.352260  0.0
2     6.833771  10.770320  0.0
3     7.349341  10.903545  0.0
4     7.442586  11.730364  0.0
...        ...        ...  ...
1995  8.580453  11.368367  1.0
1996  6.815484   8.230757  1.0
1997  7.183152   9.043539  1.0
1998  7.928611  10.058593  1.0
1999  9.009942  11.374089  1.0

[2000 rows x 3 columns]>

In [91]:
data

,x1,x2,y
0,7.606876,11.421913,0.0
1,8.300079,12.352260,0.0
2,6.833771,10.770320,0.0
3,7.349341,10.903545,0.0
4,7.442586,11.730364,0.0
...,...,...,...
1995,8.580453,11.368367,1.0
1996,6.815484,8.230757,1.0
1997,7.183152,9.043539,1.0
1998,7.928611,10.058593,1.0


In [99]:
data_numpy

array([[ 7.60687595, 11.42191282,  0.        ],
       [ 8.30007867, 12.35226012,  0.        ],
       [ 6.83377131, 10.7703199 ,  0.        ],
       ...,
       [ 7.18315177,  9.0435393 ,  1.        ],
       [ 7.92861062, 10.05859259,  1.        ],
       [ 9.00994167, 11.37408856,  1.        ]])

In [83]:
clases_unicas = np.unique(y_ent) # Clases únicas en el conjunto de entrenamiento
num_clases = clases_unicas.size # Número total de clases
num_caracteristicas = X_ent.shape[-1] # Número de características por muestra
num_muestras = X_ent.shape[0] # Número total de muestras en el conjunto de entrenamiento

prob_atributo_dado_clase = np.zeros((num_clases, num_caracteristicas)) # Parámetros condicionales: probabilidad de que cada atributo sea 1 para cada clase.
prob_clase = np.zeros(num_clases) # Probabilidades a priori: probabilidad de ocurrencia de cada clase.

for i, clase in enumerate(clases_unicas):
    # Filtra ejemplos de la clase actual
    X_clase = X_ent[y_ent == clase] # Filtra los valores en X_ent que cumplen y == clase 
    cuenta_atributos = np.count_nonzero(X_clase, axis=0) # Cuenta las veces que cada atributo es 1 en la clase actual, porque es BERNOULLI
                                                         # axis = 0 indica que se aplica a lo largo de las columnas
    num_muestras_clase = X_clase.shape[0] # Número de muestras en la clase actual
    prob_atributo_dado_clase[i, :] = cuenta_atributos / num_muestras_clase # Calcula la probabilidad de cada atributo dado la clase: P(atributo = 1 | clase)
                                                                            # [i, :] := modifica toda la fila 'i' de la matriz 
    prob_clase[i] = num_muestras_clase / num_muestras # Calcula la probabilidad a priori de la clase: P(clase)
    
def calcular_probabilidades_posteriores(self, X):
    """
    Calcula las probabilidades posteriores (proporcionales) para cada muestra:
    
    P(clase|datos) ∝ P(datos|clase) * P(clase)
    
    Se omite el denominador P(datos) ya que es constante para todas las clases.
    """
    # Inicializa la matriz de probabilidades con ceros:
    probabilidades = np.zeros((X.shape[0], self.n_clases)) # Cada fila corresponde a una muestra y cada columna a una clase.
    
    # Se itera sobre cada clase:
    # 1.- calcula la verosimilitud: P(X|C)
    # 2.- la multiplica por la probabilidad a priori: P(C)
    for indice_clase in range(self.n_clases):
        verosimilitud = np.prod(bernoulli(X, self.qa[indice_clase, :]), axis=1) #Aquí usamos la distribución de Bernoulli para modelar cada atributo independientemente.
        probabilidades[:, indice_clase] = verosimilitud * self.qc[indice_clase]
    
    return probabilidades

def predict(self, X):
    """
    Predice clases de conjunto de datos
    """
    return np.argmax(self.calcular_probabilidades_posteriores(X), axis=1)

In [97]:
class BernoulliNB:
    def fit(self, X, y):
        """
        Estima parámetros por máxima verosimilitud.
        """
        self.clases = np.unique(y)
        self.n_clases = self.clases.size
        self.n_atr = X.shape[-1]
        n = X.shape[0]

        self.qa = np.zeros((self.n_clases, self.n_atr))
        self.qc = np.zeros((self.n_clases))
        for i, c in enumerate(self.clases):
            Xc = X[y == c]
            nc = Xc.shape[0]
            cuentas = np.count_nonzero(Xc, axis=0)
            self.qa[i, :] = cuentas / nc
            self.qc[i] = nc / n

    def predict_proba(self, X):
        """
        Calcula la probabilidad a posteriori (proporcional) para cada clase.
        """
        prop = np.zeros((X.shape[0], self.n_clases))
        for i in range(self.n_clases):
            prop[:, i] = np.prod(bernoulli(X, self.qa[i, :]), axis=1) * self.qc[i]
        return prop

    def predict(self, X):
        """
        Predice las clases para el conjunto de datos X.
        """
        return np.argmax(self.predict_proba(X), axis=1)

import numpy as np

def bernoulli(x, q, epsilon=1e-9):
    """
    Función que calcula la probabilidad usando la distribución Bernoulli.
    - x: Valores (0 o 1) de las características.
    - q: Probabilidad de éxito (P(x=1)).
    Se utiliza clip para evitar problemas numéricos.
    """
    q = np.clip(q, epsilon, 1.0 - epsilon)
    return q**x * (1.0 - q)**(1.0 - x)

import pandas as pd

# Leer el archivo CSV
archivo = "aves.csv"
data = pd.read_csv(archivo, index_col=0)
data_numpy = data.to_numpy()

# Separar las características (todas menos la última) y la etiqueta (la última columna)
X_ent = data_numpy[:, :-1]
y_ent = data_numpy[:, -1]

# Ejemplo: definir datos nuevos para predecir.
# Si no tienes un conjunto separado, puedes usar el mismo X_ent o crear otro arreglo.
X_nuevos = X_ent.copy()  # Aquí se usa el mismo conjunto para la predicción.


Predicciones: [0 0 0 ... 0 0 0]
Probabilidades: [[9.07965052e+152 9.07965052e+152]
 [3.71551828e+167 3.71551828e+167]
 [1.36707114e+140 1.36707114e+140]
 ...
 [5.48516630e+127 5.48516630e+127]
 [3.83529801e+143 3.83529801e+143]
 [1.42969142e+165 1.42969142e+165]]


In [85]:
import numpy as np

class NaiveBayesBernoulli:
    def __init__(self):
        self.num_clases = None
        self.num_caracteristicas = None
        self.proba_caracteristica_dado_clase = None
        self.proba_clase = None

    def fit(self, X_ent, y_ent):
        """
        Entrena el modelo con el conjunto de entrenamiento.
        """
        clases_unicas = np.unique(y_ent)  # Clases únicas en el conjunto de entrenamiento
        self.num_clases = clases_unicas.size  # Número total de clases
        self.num_caracteristicas = X_ent.shape[-1]  # Número de características por muestra
        num_muestras = X_ent.shape[0]  # Número total de muestras en el conjunto de entrenamiento

        self.proba_caracteristica_dado_clase = np.zeros((self.num_clases, self.num_caracteristicas))  # Parámetros condicionales: probabilidad de que cada atributo sea 1 para cada clase.
        self.proba_clase = np.zeros(self.num_clases)  # Probabilidades a priori: probabilidad de ocurrencia de cada clase.

        for i, clase in enumerate(clases_unicas):
            # Filtra ejemplos de la clase actual
            X_clase = X_ent[y_ent == clase]  # Filtra los valores en X_ent que cumplen y == clase 
            cuenta_atributos = np.count_nonzero(X_clase, axis=0)  # Cuenta las veces que cada atributo es 1 en la clase actual, porque es BERNOULLI
                                                                 # axis = 0 indica que se aplica a lo largo de las columnas
            num_muestras_clase = X_clase.shape[0]  # Número de muestras en la clase actual
            self.proba_caracteristica_dado_clase[i, :] = cuenta_atributos / num_muestras_clase  # Calcula la probabilidad de cada atributo dado la clase: P(atributo = 1 | clase)
                                                                                         # [i, :] := modifica toda la fila 'i' de la matriz 
            self.proba_clase[i] = num_muestras_clase / num_muestras  # Calcula la probabilidad a priori de la clase: P(clase)
    
    def calcular_probabilidades_posteriores(self, X):
        """
        Calcula las probabilidades posteriores (proporcionales) para cada muestra:
        
        P(clase|datos) ∝ P(datos|clase) * P(clase)
        
        Se omite el denominador P(datos) ya que es constante para todas las clases.
        """
        # Inicializa la matriz de probabilidades con ceros:
        probabilidades = np.zeros((X.shape[0], self.num_clases))  # Cada fila corresponde a una muestra y cada columna a una clase.
        
        # Se itera sobre cada clase:
        # 1.- calcula la verosimilitud: P(X|C)
        # 2.- la multiplica por la probabilidad a priori: P(C)
        for indice_clase in range(self.num_clases):
            verosimilitud = np.prod(self.bernoulli(X, self.proba_caracteristica_dado_clase[indice_clase, :]), axis=1)  # Aquí usamos la distribución de Bernoulli para modelar cada atributo independientemente.
            probabilidades[:, indice_clase] = verosimilitud * self.proba_clase[indice_clase]
        
        return probabilidades

    def predict(self, X):
        """
        Predice clases de conjunto de datos
        """
        return np.argmax(self.calcular_probabilidades_posteriores(X), axis=1)
    
    @staticmethod
    def bernoulli(x, q):
      """
      Distribución de bernoulli
      -x: Matriz o vector de valores que toma la variable (0 o 1).
      -q: Vector de probabilidades asociadas a cada atributo o experimento.
      return: Si x = 1 el resultado es q.
              Si x = 0 el resultado es 1−q.
      """
      return q**x * (1.0 - q)**(1.0 - x)

In [102]:
import pandas as pd
import numpy as np

# Cargar la data
archivo = "aves.csv"
data = pd.read_csv(archivo, index_col=0)
data_numpy = data.to_numpy()

# Separar características y etiquetas
X_ent = data_numpy[:, :-1]
y_ent = data_numpy[:, -1]

# Aplicar un umbral para convertir las características en binarias.
# Usaremos la media de cada columna como umbral:
umbral = np.mean(X_ent, axis=0)
X_binaria = (X_ent >= umbral).astype(int)

# Instanciar y entrenar el clasificador
nbb = NaiveBayesBernoulli()
nbb.fit(X_binaria, y_ent)

# Realizar predicciones utilizando el mismo conjunto (puedes usar otro conjunto si lo deseas)
y_pred = nbb.predict(X_binaria)
print("Predicciones:", y_pred)

# Opcional: Calcular y mostrar las probabilidades a posteriori
probabilidades = nbb.calcular_probabilidades_posteriores(X_binaria)
probabilidades

Predicciones: [0 0 1 ... 1 1 0]


array([[0.206919, 0.040095],
       [0.208581, 0.040905],
       [0.042081, 0.207405],
       ...,
       [0.042081, 0.207405],
       [0.042081, 0.207405],
       [0.208581, 0.040905]])